# Webscraping 

In [1]:
import re
import base64
def get_ip_port(tag,p):
    scrpt=tag.find('script')  
    l=str(scrpt).split('.decode')[1].split(')')[0].lstrip('(').strip('""') 
    decoded = base64.b64decode(l)  
    ip=decoded.decode('utf8')   #ipaddress 
    return  (str(ip)+':'+str(p)) #returns ip+port

def get(row):
    k=[]
    try:
        l=row.find_all('td')
        value=[l[0],l[1],l[len(l)-1]] 
        x=get_ip_port(value[0],value[1].text)
        k.append(x)
        k.append(value[2].text) 
    except:
        pass
    
    return k #returns list containing ip+port and Last checked

In [2]:
#return a list port,last time checked
def rows_values(rows):
    data=[]
    for row in rows:
        v=get(row)
        data.append(v)     
    return data #list of list containing ip+port and Last checked


In [3]:
import requests
import time
from bs4 import BeautifulSoup
from bs4.builder import _htmlparser
import pandas as pd

def pages():
    l=[]
    for tpe in ['uptime','ping','speed']:
        for p in ['1','2','3','4','5']:
            url = f"http://free-proxy.cz/en/proxylist/country/all/https/{tpe}/all/{p}"
            try:
                page = requests.request('POST',url=url)
                page_contents=page.text
                doc= BeautifulSoup(page_contents, 'html.parser')
                table=doc.find_all('table',{'id':'proxy_list'})
                rows=table[0].find_all('tr')
                rows=rows[1:]
                val=rows_values(rows) #rows=list of all tr elements  

                df=pd.DataFrame(val,columns=['IPPort','Last Checked'])
                l.append(df)
            except:
                print("Connection refused by the server..")
                print("Let me sleep for 5 seconds")
                print("ZZzzzz...")
                time.sleep(5)
                print("Was a nice sleep, now let me continue...")
                continue
    return l #list of dataframe
list_of_df=pages()

In [4]:
import pandas as pd
df=pd.concat(list_of_df,ignore_index=True) #concatenates the dataframe
print(len(df))

df=df[df.astype(str)['IPPort'] != '[]'] 
result=df.dropna() #removes none values 

result.sample(5)

540


,IPPort,Last Checked
338,118.99.124.172:8080,16 hours ago
3,199.19.95.130:1080,9 hours ago
39,190.109.0.229:999,15 hours ago
516,20.206.99.104:3128,8 hours ago
296,139.99.74.79:8080,8 hours ago


In [5]:
from datetime import datetime, timedelta
k=[]
for index, row in result.iterrows():
    i=row["Last Checked"]
    if i.split(' ')[1]=='days' and int(i.split(' ')[0])>=2:
        result=result.drop(index)                               #drops row with last checked more than 2 days 
    elif i.split(' ')[1]=='hours':   #for hours ago 
        h=int(i.split(' ')[0])
        hh=datetime.now()- timedelta(hours =h)
        k.append(int(hh.timestamp()))
    elif i.split(' ')[1]=='minutes':  #for minutes ago
        m=int(i.split(' ')[0])
        mm=datetime.now()- timedelta(minutes =m)
        k.append(int(hh.timestamp()))
    elif i.split(' ')[1]=='seconds': #for seconds ago
        s=int(i.split(' ')[0])
        ss=datetime.now()- timedelta(seconds =s)
        k.append(int(ss.timestamp()))  
result['Epoch']=k        

<ipython-input-5-47faf94b11a7>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Epoch']=k


In [6]:
print(result.sample(5)) #with new column epoch 


                   IPPort  Last Checked       Epoch
43    186.95.134.149:9999  10 hours ago  1630384308
420     18.230.101.196:80  13 hours ago  1630373509
94    36.67.152.213:11111  15 hours ago  1630366308
176  187.190.115.139:9991  16 hours ago  1630362708
389    68.180.3.212:45554   6 hours ago  1630398709


In [7]:
result.to_csv(' Data.csv',encoding = 'utf-8') #saving to csv 

## Insert/update in the database

In [8]:
import mysql.connector
from mysql.connector import Error
try:
    conn=mysql.connector.connect(host="68.183.179.154",user="1688dev2",password="D#v1688RTF",database="1688_db",charset="utf8")
    cur=conn.cursor(prepared=True)
    for (row,rs) in result.iterrows():
        IP=rs[0]
        lct=rs[2]
        query="insert into proxy(IPPort,last_checked_time) values(%s,%s) on duplicate key update  IPPort=values(IPPort)";
        #inserts into the database simultanously updates if duplicates found
        cur.execute(query,(IP,lct))
    conn.commit()
    cur.close()
except Error as e:
    print("error in mysql connection =",e)
finally :
    conn.close()
    